#  PikaPikaGenerator - Interactive Demo
# 
**Progetto:** Generative Synthesis of Pokémon Sprites from Textual Descriptions  
 **Corso:** Deep Learning - Politecnico di Bari  
 **Studente:** Pasquale Alessandro Denora  
 **Professore:** Vito Walter Anelli 

#  Import e Setup Iniziale
 
 Il file inizia importando tutte le librerie necessarie per creare l'interfaccia demo:
 - **gradio**: Framework per creare interfacce web interattive
 - **torch**: Per gestire il modello e le operazioni tensori
 - **PIL**: Per manipolazione immagini generate
 - **yaml & json**: Per configurazione e serializzazione dati
 - **pathlib**: Per gestione paths in modo object-oriented
 - **typing**: Per type hints e migliore documentazione
 
 **Import specifici del progetto**:
 - `src.models.architecture`: Per creare e gestire il modello
 - `src.utils.visualization`: Per heatmap attention e plot metriche


In [ ]:
import gradio as gr
import torch
import numpy as np
from PIL import Image
import yaml
import json
from pathlib import Path
import logging
from typing import Tuple, List, Optional

from src.models.architecture import create_model
from src.utils.visualization import create_attention_heatmap, plot_metrics

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


#  Classe PikaPikaGeneratorApp - Inizializzazione
 
 La classe principale `PikaPikaGeneratorApp` (righe 18-214) gestisce tutta la logica della demo:
 
 **Inizializzazione** (righe 21-31):
 - Carica configurazione da file YAML specificato
 - Determina device (CPU/GPU) automaticamente
 - Inizializza attributi per modello e stato caricamento
 - Chiama `load_model()` per caricare modello addestrato
 
 **Attributi principali**:
 - `self.config`: Configurazione completa del progetto
 - `self.device`: Device per inferenza (CPU/GPU)
 - `self.model`: Modello PikaPikaGenerator caricato
 - `self.model_loaded`: Flag per verificare se modello è pronto
 
 **Parametri config utilizzati**: Tutto il file config.yaml viene caricato per consistency


In [ ]:
class PikaPikaGeneratorApp:
    """Classe principale applicazione per l'Interfaccia Gradio di PikaPikaGenerator"""
    
    def __init__(self, config_path: str = "configs/config.yaml"):
        # Carica la configurazione
        with open(config_path, 'r') as f:
            self.config = yaml.safe_load(f)
        
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = None
        self.model_loaded = False
        
        # Carica il modello
        self.load_model()

#  Caricamento del Modello Addestrato
 
 Il metodo `load_model()` (righe 33-57) carica il modello addestrato per la demo:
 
 **Strategia di caricamento** (righe 35-53):
 - Cerca prima `best_model.pt` (modello con migliori performance)
 - Path costruito da `config['paths']['checkpoints_dir']`
 - Se esiste, procede con caricamento
 - Se non esiste, imposta `model_loaded = False`
 
 **Processo di caricamento** (righe 42-50):
 - Crea modello usando `create_model(config)`
 - Carica checkpoint salvato durante training
 - Restore dei pesi con `load_state_dict()`
 - Imposta modello in `eval()` mode per inferenza
 - Aggiorna flag `model_loaded = True`
 
 **Error handling robusto** (righe 54-57):
 - Try/catch per gestire problemi di caricamento
 - Warning se modello non trovato
 - Error se caricamento fallisce
 - Flag sempre aggiornato correttamente


In [ ]:
def load_model(self):
        """Carica il modello addestrato"""
        try:
            model_path = Path(self.config['paths']['checkpoints_dir']) / 'best_model.pt'
            
            if model_path.exists():
                logger.info(f"Loading model from {model_path}")
                
                # Crea il modello
                self.model = create_model(self.config).to(self.device)
                
                # Carica il checkpoint
                checkpoint = torch.load(model_path, map_location=self.device)
                self.model.load_state_dict(checkpoint['model_state_dict'])
                self.model.eval()
                
                self.model_loaded = True
                logger.info("Model loaded successfully!")
            else:
                logger.warning(f"Model not found at {model_path}")
                self.model_loaded = False
                
        except Exception as e:
            logger.error(f"Error loading model: {e}")
            self.model_loaded = False

#  Generazione Singola Sprite con Attention
 
 Il metodo `generate_sprite()` (righe 59-95) implementa la generazione singola di sprite:
 
 **Input parameters**:
 - `description`: Testo descrittivo Pokemon
 - `noise_scale`: Scala del rumore casuale (default 1.0)
 - `show_attention`: Flag per mostrare attention heatmap
 
 **Validazione input** (righe 64-71):
 - Controlla se modello è caricato correttamente
 - Verifica che description non sia vuota
 - Return messaggi di errore user-friendly se problemi
 
 **Processo di generazione** (righe 73-87):
 - Crea rumore casuale con scala specificata
 - Chiama `model.generate()` per ottenere sprite
 - Converte tensor output in PIL Image
 - Opzionalmente genera attention heatmap per interpretabilità
 
 **Output triplo**:
 - `sprite_img`: Immagine Pokemon generata
 - `attention_img`: Heatmap attention (se richiesta)
 - `status`: Messaggio di stato per utente


In [ ]:
def generate_sprite(
        self,
        description: str,
        noise_scale: float = 1.0,
        show_attention: bool = False
    ) -> Tuple[Image.Image, Optional[Image.Image], str]:
        """Generate a Pokemon sprite from description"""
        
        if not self.model_loaded:
            return None, None, "❌ Model not loaded. Please train the model first."
        
        if not description.strip():
            return None, None, "❌ Please enter a description."
        
        try:
            # Genera sprite
            noise = torch.randn(1, self.config['model']['generator']['noise_dim']) * noise_scale
            noise = noise.to(self.device)
            
            sprite = self.model.generate(description, noise=noise, device=self.device)
            sprite_img = Image.fromarray(sprite)
            
            # Genera attenzione heatmap se richiesto
            attention_img = None
            if show_attention:
                _, tokens, attention_weights = self.model.get_attention_visualization(
                    description, device=self.device
                )
                attention_img = create_attention_heatmap(tokens, attention_weights)
            
            status = f"✅ Generated sprite for: {description[:50]}..."
            
            return sprite_img, attention_img, status
            
        except Exception as e:
            logger.error(f"Error generating sprite: {e}")
            return None, None, f"❌ Error: {str(e)}"

#  Generazione Batch con Variazioni Multiple
 
 Il metodo `batch_generate()` (righe 97-144) permette di generare multiple sprite con variazioni:
 
 **Input processing** (righe 102-112):
 - Parse delle descrizioni (una per linea)
 - Filtra linee vuote con list comprehension
 - Verifica che ci sia almeno una descrizione valida
 - Return early se input non valido
 
 **Loop di generazione** (righe 116-124):
 - Loop nidificato: per ogni descrizione, genera N variazioni
 - Rumore diverso per ogni variazione (casualità controllata)
 - Genera sprite chiamando `model.generate()`
 - Converte ogni output in PIL Image
 
 **Annotazione immagini** (righe 127-135):
 - Usa PIL.ImageDraw per aggiungere testo
 - Tenta di caricare font arial, fallback a default
 - Aggiunge nome descrizione + numero variazione
 - Testo con outline per visibilità su qualsiasi background
 
 **Output finale**:
 - Lista di immagini generate con annotazioni
 - Status message con count totale di sprite generate

In [ ]:

def batch_generate(
        self,
        descriptions: str,
        num_variations: int = 3,
        noise_scale: float = 1.0
    ) -> Tuple[List[Image.Image], str]:
        """Genera Sprite Multiple per Descrizioni Multiple"""
        
        if not self.model_loaded:
            return [], "❌ Model not loaded."
        
        # Parse descriptions (one per line)
        desc_list = [d.strip() for d in descriptions.strip().split('\n') if d.strip()]
        
        if not desc_list:
            return [], "❌ Please enter at least one description."
        
        generated_images = []
        
        try:
            for desc in desc_list:
                for i in range(num_variations):
                    # Rumore differente per ogni variazione
                    noise = torch.randn(1, self.config['model']['generator']['noise_dim']) * noise_scale
                    noise = noise.to(self.device)
                    
                    sprite = self.model.generate(desc, noise=noise, device=self.device)
                    sprite_img = Image.fromarray(sprite)
                    
                    # Aggiungi testo alla sprite
                    from PIL import ImageDraw, ImageFont
                    draw = ImageDraw.Draw(sprite_img)
                    try:
                        font = ImageFont.truetype("arial.ttf", 10)
                    except:
                        font = ImageFont.load_default()
                    
                    text = f"{desc[:30]}... (v{i+1})"
                    draw.text((5, 5), text, fill='white', font=font, stroke_width=1, stroke_fill='black')
                    
                    generated_images.append(sprite_img)
            
            status = f"✅ Generated {len(generated_images)} sprites from {len(desc_list)} descriptions"
            return generated_images, status
            
        except Exception as e:
            logger.error(f"Error in batch generation: {e}")
            return [], f"❌ Error: {str(e)}"

#  Caricamento Metriche di Valutazione
 
 Il metodo `load_evaluation_metrics()` (righe 146-176) carica e visualizza le metriche del modello:
 
 **Ricerca checkpoint** (righe 149-154):
 - Scansiona directory checkpoints per trovare file disponibili
 - Pattern `checkpoint_epoch_*.pt` per tutti i checkpoint
 - Se nessun checkpoint trovato, return messaggio di errore
 - Strategia robusta che non assume presenza di file specifici
 
 **Selezione ultimo checkpoint** (righe 156-158):
 - Usa `max()` con `key=lambda` per trovare file più recente
 - Basato su timestamp di modifica (`stat().st_mtime`)
 - Carica checkpoint più recente disponibile
 
 **Estrazione e visualizzazione metriche** (righe 160-172):
 - Verifica presenza di `val_metrics` nel checkpoint
 - Usa `plot_metrics()` da utils.visualization per grafico
 - Formatta metriche come testo strutturato per display
 - Return sia immagine che testo per doppia visualizzazione
 
 **Error handling**: Try/catch completo per gestire file mancanti o corrotti


In [ ]:
def load_evaluation_metrics(self) -> Tuple[Optional[Image.Image], str]:
        """Load and display evaluation metrics"""
        
        try:
            # Carica metriche dall'ultimo checkpoint
            checkpoint_dir = Path(self.config['paths']['checkpoints_dir'])
            checkpoint_files = list(checkpoint_dir.glob('checkpoint_epoch_*.pt'))
            
            if not checkpoint_files:
                return None, "❌ No checkpoints found."
            
            # Ottieni l'ultimo checkpoint
            latest_checkpoint = max(checkpoint_files, key=lambda p: p.stat().st_mtime)
            checkpoint = torch.load(latest_checkpoint, map_location='cpu')
            
            if 'val_metrics' in checkpoint:
                metrics = checkpoint['val_metrics']
                metrics_img = plot_metrics(metrics, f"Validation Metrics - Epoch {checkpoint['epoch']}")
                
                # Formatta le metriche come testo
                metrics_text = f"**Epoch {checkpoint['epoch']} Metrics:**\n"
                for key, value in metrics.items():
                    metrics_text += f"- {key}: {value:.4f}\n"
                
                return metrics_img, metrics_text
            else:
                return None, "❌ No validation metrics found in checkpoint."
                
        except Exception as e:
            logger.error(f"Error loading metrics: {e}")
            return None, f"❌ Error: {str(e)}"

##  Informazioni del Modello
 
 Il metodo `get_model_info()` (righe 178-214) fornisce informazioni dettagliate sul modello:
 
 **Conteggio parametri** (righe 186-187):
 - `sum(p.numel() for p in model.parameters())`: Conta tutti i parametri
 - `sum(... if p.requires_grad)`: Conta solo parametri trainable
 - Informazione cruciale per capire complessità modello
 
 **Informazioni architettura** (righe 190-208):
 - **Text Encoder**: Nome modello BERT e dimensione hidden
 - **Generator**: Canali base e dimensione output finale
 - **Training config**: Batch size, learning rate, pesi delle loss
 - Tutte le info estratte dalla configurazione caricata
 
 **Formatting user-friendly** (righe 190-208):
 - Usa f-strings per formattazione pulita
 - Numeri formattati con comma separators 
 - Struttura gerarchica con indentazione
 - Markdown formatting per display ricco in Gradio
 
 **Parametri config utilizzati**:
 - `model.encoder.model_name`, `hidden_dim`, `max_length`
 - `model.generator.base_channels`, `output_size`  
 - `training.batch_size`, `learning_rate`
 - `loss` weights per reconstruction, perceptual, adversarial


In [ ]:
def get_model_info(self) -> str:
        
        
        if not self.model_loaded:
            return "❌ Model not loaded."
        
        try:
            # Conta i parametri del modello
            total_params = sum(p.numel() for p in self.model.parameters())
            trainable_params = sum(p.numel() for p in self.model.parameters() if p.requires_grad)
            
            info = f"""
            **Model Architecture:**
            - Text Encoder: {self.config['model']['encoder']['model_name']}
            - Hidden Dimension: {self.config['model']['encoder']['hidden_dim']}
            - Generator Base Channels: {self.config['model']['generator']['base_channels']}
            - Output Size: {self.config['model']['generator']['output_size']}x{self.config['model']['generator']['output_size']}
            
            **Parameters:**
            - Total: {total_params:,}
            - Trainable: {trainable_params:,}
            
            **Training Configuration:**
            - Batch Size: {self.config['training']['batch_size']}
            - Learning Rate: {self.config['training']['learning_rate']}
            - Loss Weights:
              - Reconstruction: {self.config['loss']['reconstruction_weight']}
              - Perceptual: {self.config['loss']['perceptual_weight']}
              - Adversarial: {self.config['loss']['adversarial_weight']}
            
            **Device:** {self.device}
            """
            
            return info
            
        except Exception as e:
            return f"❌ Error getting model info: {str(e)}"

#  Creazione Interfaccia Gradio - Setup Principale

 La funzione `create_interface()` (righe 217-378) crea l'interfaccia web completa:
 
 **Inizializzazione app** (righe 220-234):
 - Istanzia `PikaPikaGeneratorApp()` che carica tutto
 - Setup CSS customizzato per styling avanzato
 - Definisce stili per header banner e layout generale
 
 **CSS Styling** (righe 224-234):
 - Font family professionale: Segoe UI, Tahoma, Geneva
 - Header banner con gradient background (purple to blue)
 - Padding, border-radius per estetica moderna
 - Text alignment e colors coordinate
 
 **Gradio Blocks setup** (righe 237-248):
 - `gr.Blocks()` per layout customizzato avanzato
 - CSS applicato e title impostato
 - Header HTML con branding del progetto
 - Subtitle che spiega functionality e tecnologie

In [ ]:
def create_interface():
    """Crea l'interfaccia Gradio"""
    
    app = PikaPikaGeneratorApp()
    
    # Stile CSS per l'interfaccia
    css = """
    .gradio-container {
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .header-banner {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 2rem;
        border-radius: 1rem;
        text-align: center;
        color: white;
        margin-bottom: 2rem;
    }
    """
    
    with gr.Blocks(css=css, title="PikaPikaGenerator - Advanced Pokemon Sprite Generation") as demo:
        
        # Header
        gr.HTML("""
        <div class="header-banner">
            <h1> PikaPikaGenerator </h1>
            <h3>Advanced Text-to-Pokemon Sprite Generation</h3>
            <p>Using Encoder-Decoder Architecture with Attention Mechanism</p>
        </div>
        """)

#  Tab "Generate Sprite" - Generazione Singola
 
 **Primo tab principale** (righe 248-275): Interface per generazione singola sprite
 
 **Layout a colonne** (righe 251-275):
 - **Colonna sinistra (scale=2)**: Input controls più larghi
   - `gr.Textbox()` per descrizione Pokemon (3 linee)
   - `gr.Slider()` per noise scale (0.1-2.0, default 1.0)
   - `gr.Checkbox()` per abilitare attention heatmap
   - `gr.Button()` principale per generazione
 
 - **Colonna destra (scale=1)**: Output display
   - `gr.Image()` per sprite generata
   - `gr.Image()` per attention heatmap (hidden di default)
   - `gr.Textbox()` per status messages
 
 **Esempi predefiniti** (righe 278-291):
 - 8 descrizioni Pokemon diverse per testing rapido
 - Coprono diversi tipi: electric, water, fire, grass, ghost, fairy, steel, ice
 - Pattern realistico che utenti potrebbero usare
 - `gr.Examples()` collega examples all'input textbox



In [ ]:
with gr.Tabs():
            
            # Single Generation Tab
            with gr.TabItem(" Generate Sprite"):
                with gr.Row():
                    with gr.Column(scale=2):
                        description_input = gr.Textbox(
                            label="Pokemon Description",
                            placeholder="Enter a detailed Pokemon description...",
                            lines=3
                        )
                        
                        with gr.Row():
                            noise_scale = gr.Slider(
                                minimum=0.1, maximum=2.0, value=1.0, step=0.1,
                                label="Noise Scale (Variation)"
                            )
                            show_attention = gr.Checkbox(
                                label="Show Attention Heatmap",
                                value=False
                            )
                        
                        generate_btn = gr.Button(" Generate Sprite", variant="primary", size="lg")
                    
                    with gr.Column(scale=1):
                        generated_image = gr.Image(label="Generated Sprite", type="pil")
                        attention_heatmap = gr.Image(label="Attention Heatmap", type="pil", visible=False)
                        status_output = gr.Textbox(label="Status", interactive=False)
                
                # Esempi
                gr.Examples(
                    examples=[
                        ["A small yellow electric mouse Pokemon with red cheeks and a lightning bolt shaped tail"],
                        ["A large blue turtle Pokemon with water cannons protruding from its shell"],
                        ["An orange dragon-type Pokemon with wings and a flame on its tail"],
                        ["A pink fairy Pokemon with ribbons and a sweet expression"],
                        ["A dark ghost Pokemon with purple flames and menacing red eyes"],
                        ["A steel-type bird Pokemon with sharp metallic feathers"],
                        ["A grass Pokemon that looks like a walking tree with leaves for hands"],
                        ["An ice-type Pokemon resembling a crystalline wolf with frozen breath"]
                    ],
                    inputs=[description_input],
                    label="Example Descriptions:"
                )

# Tab "Batch Generation" - Generazione Multiple
 
 **Secondo tab principale** (righe 294-323): Interface per generazione batch
 
 **Input area** (righe 296-313):
 - `gr.Textbox()` con 8 linee per multiple descriptions
 - Placeholder che spiega formato "one per line"
 - `gr.Slider()` per numero variazioni per descrizione (1-5)
 - `gr.Slider()` per noise scale globale
 - `gr.Button()` per avviare batch generation
 
 **Output area** (righe 315-323):
 - `gr.Gallery()` per mostrare tutte le immagini generate
 - `show_label=True` per chiarezza
 - Layout griglia: 4 colonne, 2 righe
 - `height="auto"` per scrolling se necessario
 - `gr.Textbox()` per status batch operation
 
 **Vantaggi batch mode**:
 - Genera multiple Pokemon simultaneamente
 - Variazioni multiple per ogni descrizione
 - Efficient per testing o confronti
 - Gallery view per vedere tutto insieme


In [ ]:
# Batch Generation Tab
with gr.TabItem(" Batch Generation"):
                with gr.Row():
                    with gr.Column():
                        batch_descriptions = gr.Textbox(
                            label="Pokemon Descriptions (one per line)",
                            placeholder="Enter multiple descriptions, one per line...",
                            lines=8
                        )
                        
                        with gr.Row():
                            num_variations = gr.Slider(
                                minimum=1, maximum=5, value=3, step=1,
                                label="Variations per Description"
                            )
                            batch_noise_scale = gr.Slider(
                                minimum=0.1, maximum=2.0, value=1.0, step=0.1,
                                label="Noise Scale"
                            )
                        
                        batch_generate_btn = gr.Button(" Generate Batch", variant="primary")
                    
                    with gr.Column():
                        batch_gallery = gr.Gallery(
                            label="Generated Sprites",
                            show_label=True,
                            columns=4,
                            rows=2,
                            height="auto"
                        )
                        batch_status = gr.Textbox(label="Status", interactive=False)
            

#  Tab "Model Info & Metrics" - Informazioni e Metriche
 
 **Terzo tab principale** (righe 326-335): Interface per info modello e metriche
 
 **Layout a due colonne**:
 
 **Colonna sinistra** (righe 328-331):
 - `gr.Button()` per caricare metriche di valutazione
 - `gr.Image()` per visualizzare plot delle metriche
 - `gr.Markdown()` per testo formattato delle metriche
 
 **Colonna destra** (righe 333-335):
 - `gr.Button()` per ottenere info del modello
 - `gr.Markdown()` per display info architettura
 
 **Funzionalità**:
 - **Load Metrics**: Carica ultimo checkpoint e mostra performance
 - **Model Info**: Mostra architettura, parametri, configurazione training
 - **Visual + Text**: Doppia rappresentazione per completeness
 - **Markdown support**: Rich formatting per info strutturate


In [ ]:
# Model Info Tab
with gr.TabItem(" Model Info & Metrics"):
                with gr.Row():
                    with gr.Column():
                        info_btn = gr.Button(" Load Evaluation Metrics", variant="primary")
                        metrics_plot = gr.Image(label="Metrics Visualization", type="pil")
                        metrics_text = gr.Markdown()
                    
                    with gr.Column():
                        model_info_btn = gr.Button(" Get Model Info", variant="primary")
                        model_info_display = gr.Markdown()

#  Event Handlers e Interazioni
 
 **Footer HTML** (righe 338-343): Footer informativo del progetto
 
 **Event handlers setup** (righe 346-377): Connessioni tra UI e funzioni backend
 
 **Dynamic visibility** (righe 346-347):
 - Funzione `update_attention_visibility()` per show/hide attention heatmap
 - Collegata al checkbox `show_attention`
 - Return `gr.update(visible=show)` per controllo dinamico UI
 
 **Main button connections**:
 - **Generate button**: Collega a `app.generate_sprite()` con 3 input → 3 output
 - **Batch button**: Collega a `app.batch_generate()` con 3 input → 2 output
 - **Metrics button**: Collega a `app.load_evaluation_metrics()` → 2 output
 - **Info button**: Collega a `app.get_model_info()` → 1 output
 
 **Return demo object**: Interfaccia completa pronta per lancio


In [ ]:

        # Event handlers
        def update_attention_visibility(show):
            return gr.update(visible=show)
        
        show_attention.change(
            fn=update_attention_visibility,
            inputs=[show_attention],
            outputs=[attention_heatmap]
        )
        
        generate_btn.click(
            fn=app.generate_sprite,
            inputs=[description_input, noise_scale, show_attention],
            outputs=[generated_image, attention_heatmap, status_output]
        )
        
        batch_generate_btn.click(
            fn=app.batch_generate,
            inputs=[batch_descriptions, num_variations, batch_noise_scale],
            outputs=[batch_gallery, batch_status]
        )
        
        info_btn.click(
            fn=app.load_evaluation_metrics,
            outputs=[metrics_plot, metrics_text]
        )
        
        model_info_btn.click(
            fn=app.get_model_info,
            outputs=[model_info_display]
        )
    
    return demo

# Funzione Main - Lancio della Demo
 
 La funzione `main()` (righe 380-395) avvia effettivamente la demo Gradio:
 
 **Setup finale** (righe 382-384):
 - Log message per indicare startup
 - Chiama `create_interface()` per ottenere demo object
 - Tutto è pronto per il lancio
 
 **Configurazione lancio** (righe 386-391):
 - `server_name="0.0.0.0"`: Accessibile da qualsiasi IP (non solo localhost)
 - `server_port=7860`: Porta standard per demo Gradio
 - `share=False`: Non crea tunnel pubblico (locale only)
 - `show_error=True`: Mostra errori completi per debug
 
 **Entry point** (righe 394-395):
 - `if __name__ == "__main__"`: Permette esecuzione diretta file
 - Chiama `main()` per avviare demo standalone
 
 **Utilizzo**: `python app.py` per lanciare demo indipendentemente

In [ ]:
def main():
    """Avvia l'app Gradio"""
    logger.info("Starting PikaPikaGenerator Gradio App...")
    
    demo = create_interface()
    
    demo.launch(
        server_name="0.0.0.0",
        server_port=7860,
        share=False,
        show_error=True
    )


if __name__ == "__main__":
    main()

#  Riepilogo Parametri Config Utilizzati
 
 Il file `app.py` utilizza questi parametri dal file `config.yaml`:
 
 **Sezione `paths`**:
 - `checkpoints_dir`: Directory dei modelli salvati ("data/models/checkpoints_hq_v2")
 - Utilizzato per caricare `best_model.pt` e checkpoint per metriche

 **Sezione `model.generator`**:
 - `noise_dim`: Dimensione vettore rumore (256)
 - `base_channels`: Canali base generatore (768)
 - `output_size`: Dimensione finale sprite (320x320)
 - Utilizzati per info modello e generazione

 **Sezione `model.encoder`**:
 - `model_name`: Nome tokenizer BERT ("prajjwal1/bert-mini")
 - `hidden_dim`: Dimensione features testuali (512)
 - `max_length`: Lunghezza massima sequenze (128)
 - Utilizzati per info architettura
 
 **Sezione `training`**:
 - `batch_size`: Dimensione batch (2)
 - `learning_rate`: Learning rate (0.0001)
 - Utilizzati per display configurazione training
 
 **Sezione `loss`**:
 - `reconstruction_weight`: Peso L1 loss (15.0)
 - `perceptual_weight`: Peso LPIPS loss (2.5)
 - `adversarial_weight`: Peso adversarial loss (0.5)
 - Utilizzati per info training configuration
 
#  Conclusioni - Demo Interattiva PikaPikaGenerator
 
 Il file `app.py` implementa una demo Gradio completa e professionale:
 
  **Caratteristiche Principali**:
 1. **Interfaccia Multi-Tab**: Organizzazione logica delle funzionalità
 2. **Single Generation**: Generazione sprite con attention visualization
 3. **Batch Generation**: Multiple sprite con variazioni controllate
 4. **Model Analytics**: Info architettura e metriche performance
 5. **Professional UI**: CSS custom, esempi predefiniti, styling moderno
 
  **Funzionalità Avanzate**:
 - **Attention Heatmap**: Visualizzazione interpretabilità modello
 - **Noise Scale Control**: Controllo variabilità generazione
 - **Batch Processing**: Efficient generation di multiple sprite
 - **Metrics Visualization**: Monitoraggio performance modello
 - **Error Handling**: Gestione robusta di tutti gli edge case
 
  **User Experience**:
 - **Esempi predefiniti**: 8 descrizioni Pokemon ready-to-use
 - **Real-time feedback**: Status messages per ogni operazione
 - **Gallery view**: Visualizzazione organizzata per batch results
 - **Responsive design**: Layout adattivo per diverse screen size
 - **Rich formatting**: Markdown support per info strutturate

  **Integration nel Progetto**:
 - **Config consistency**: Stessa configurazione del training
 - **Model compatibility**: Carica checkpoint salvati dal trainer
 - **Utils integration**: Usa funzioni visualization per plots
 - **Standalone operation**: Può essere lanciata indipendentemente
 
  **Aspetti Tecnici**:
- **Memory efficient**: Proper device management e cleanup
- **Error resilient**: Fallback graceful per problemi modello
- **Configurable**: Tutti i parametri leggibili da config.yaml
- **Production ready**: Logging appropriato e error handling

 La demo PikaPikaGenerator offre un'**esperienza utente completa** per interagire con il modello addestrato, perfetta per showcase, testing e valutazione qualitativa! 
 
#  **Requisiti per Esecuzione**:
 - Modello addestrato salvato come `best_model.pt`
 - Configurazione `config.yaml` con parametri corretti
 - Gradio installato (`pip install gradio`)
 - Esecuzione: `python app.py` → Demo disponibile su http://localhost:7860